In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
# Đường dẫn của data
file_name = r'D:\Learning\ML\BTL\data.json'

In [3]:
# Hàm flatten dict
def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [4]:
# Mở file
with open(file_name, 'r', encoding='utf-8') as file:
    data = json.load(file)

### 1. Chênh lệch Lượng vàng chênh lệch giữa xanh và đỏ mỗi phút

In [5]:
# Hàm tính lượng vàng chênh lệch giữa đội xanh và đỏ mỗi phút
def get_gold_difference_between_blue_and_red_team_per_minute(data):
    gold_difference_df = {}
    for minute, frame in enumerate(data['info']['frames'], start=0):
        total_gold_blue = 0
        total_gold_red = 0
        for participant_id, data in frame["participantFrames"].items():
            participant_id = int(participant_id)
            total_gold = data["totalGold"]
            
            if 1 <= participant_id <= 5:
                total_gold_blue += total_gold
            elif 6 <= participant_id <= 10:
                total_gold_red += total_gold
        gold_difference_df[f"gold_difference_at_{minute}"] = total_gold_blue - total_gold_red
    return gold_difference_df


In [6]:
# Dict Lượng vàng chênh lệch giữa xanh và đỏ mỗi phút
gold_difference_df = get_gold_difference_between_blue_and_red_team_per_minute(data)
gold_difference_df

{'gold_difference_at_0': 0,
 'gold_difference_at_1': 0,
 'gold_difference_at_2': -62,
 'gold_difference_at_3': 882,
 'gold_difference_at_4': 860,
 'gold_difference_at_5': 873,
 'gold_difference_at_6': 626,
 'gold_difference_at_7': -530,
 'gold_difference_at_8': -391,
 'gold_difference_at_9': -324,
 'gold_difference_at_10': 250,
 'gold_difference_at_11': 817,
 'gold_difference_at_12': -942,
 'gold_difference_at_13': -648,
 'gold_difference_at_14': -394,
 'gold_difference_at_15': 481,
 'gold_difference_at_16': 952,
 'gold_difference_at_17': -237,
 'gold_difference_at_18': 1241,
 'gold_difference_at_19': 3676,
 'gold_difference_at_20': 3956,
 'gold_difference_at_21': 2470,
 'gold_difference_at_22': 2574,
 'gold_difference_at_23': 1554,
 'gold_difference_at_24': 1833,
 'gold_difference_at_25': 1054,
 'gold_difference_at_26': -62,
 'gold_difference_at_27': -2711,
 'gold_difference_at_28': -2418,
 'gold_difference_at_29': -3631,
 'gold_difference_at_30': -5268,
 'gold_difference_at_31': -436

### 2. Trạng thái trụ mỗi đội, mỗi loại, mỗi phút

In [7]:
# Hàm trạng thái trụ mỗi đội, mỗi loại, mỗi phút
def calculate_turret_status(data):
    initial_turrets = {
        "blue": {
            "MID_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1, "NEXUS_TURRET": 2},
            "TOP_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
            "BOT_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
        },
        "red": {
            "MID_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1, "NEXUS_TURRET": 2},
            "TOP_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
            "BOT_LANE": {"INNER_TURRET": 1, "OUTER_TURRET": 1, "BASE_TURRET": 1},
        }
    }

    # Kết quả lưu trạng thái trụ theo từng phút
    turret_status = {}

    # Lấy danh sách các frame (tương ứng phút)
    frames = data.get("info", {}).get("frames", [])

    # Duyệt qua từng frame (mỗi frame là 1 phút)
    for frame_index, frame in enumerate(frames):
        minute = frame_index

        # Duyệt qua các lane và các loại trụ để sao chép trạng thái ban đầu cho phút đầu tiên
        for team in ["blue", "red"]:
            for lane, turrets in initial_turrets[team].items():
                for tower_type, count in turrets.items():
                    # Cập nhật trạng thái trụ cho phút hiện tại
                    key = f"{team}_{lane}_{tower_type}_at_{minute}"
                    turret_status[key] = count

        # Lấy danh sách sự kiện trong frame
        events = frame.get("events", [])
        for event in events:
            # Kiểm tra nếu sự kiện là phá hủy trụ
            if event.get("type") == "BUILDING_KILL" and event.get("towerType") and event.get("laneType"):
                team_id = event.get("teamId")
                tower_type = event.get("towerType")
                lane_type = event.get("laneType")

                # Xác định đội (100 = blue, 200 = red)
                team = "blue" if team_id == 100 else "red"

                # Xác định key cho trụ bị phá hủy
                key = f"{team}_{lane_type}_{tower_type}_at_{minute}"

                # Giảm số lượng trụ trong lane và loại trụ tương ứng
                if key in turret_status:
                    turret_status[key] -= 1
                    initial_turrets[team][lane_type][tower_type] -= 1

    return turret_status


In [8]:
# Dict trạng thái trụ mỗi đội, mỗi loại, mỗi phút
remaining_turret_df = calculate_turret_status(data)
remaining_turret_df

{'blue_MID_LANE_INNER_TURRET_at_0': 1,
 'blue_MID_LANE_OUTER_TURRET_at_0': 1,
 'blue_MID_LANE_BASE_TURRET_at_0': 1,
 'blue_MID_LANE_NEXUS_TURRET_at_0': 2,
 'blue_TOP_LANE_INNER_TURRET_at_0': 1,
 'blue_TOP_LANE_OUTER_TURRET_at_0': 1,
 'blue_TOP_LANE_BASE_TURRET_at_0': 1,
 'blue_BOT_LANE_INNER_TURRET_at_0': 1,
 'blue_BOT_LANE_OUTER_TURRET_at_0': 1,
 'blue_BOT_LANE_BASE_TURRET_at_0': 1,
 'red_MID_LANE_INNER_TURRET_at_0': 1,
 'red_MID_LANE_OUTER_TURRET_at_0': 1,
 'red_MID_LANE_BASE_TURRET_at_0': 1,
 'red_MID_LANE_NEXUS_TURRET_at_0': 2,
 'red_TOP_LANE_INNER_TURRET_at_0': 1,
 'red_TOP_LANE_OUTER_TURRET_at_0': 1,
 'red_TOP_LANE_BASE_TURRET_at_0': 1,
 'red_BOT_LANE_INNER_TURRET_at_0': 1,
 'red_BOT_LANE_OUTER_TURRET_at_0': 1,
 'red_BOT_LANE_BASE_TURRET_at_0': 1,
 'blue_MID_LANE_INNER_TURRET_at_1': 1,
 'blue_MID_LANE_OUTER_TURRET_at_1': 1,
 'blue_MID_LANE_BASE_TURRET_at_1': 1,
 'blue_MID_LANE_NEXUS_TURRET_at_1': 2,
 'blue_TOP_LANE_INNER_TURRET_at_1': 1,
 'blue_TOP_LANE_OUTER_TURRET_at_1': 1,
 'b

### 3. Chênh lệch cấp độ mỗi đường, mỗi phút

In [9]:
# Hàm tính chênh lệch cấp độ mỗi đường giữa xanh và đỏ mỗi phút
def calculate_level_difference_by_lane(data):
    # Lưu trữ chênh lệch cấp độ mỗi đường, mỗi phút
    level_diff = {}

    # Lấy danh sách các frame (tương ứng phút)
    frames = data.get("info", {}).get("frames", [])

    # Định nghĩa các participantId cho các đường
    lane_participants = {
        "TOP": [1, 6],
        "JUNGLE": [2, 7],
        "MID": [3, 8],
        "BOT": [4, 9],
        "SUPPORT": [5, 10]
    }

    # Duyệt qua từng frame (mỗi frame là 1 phút)
    for frame_index, frame in enumerate(frames):
        minute = frame_index
        blue_team_levels = []
        red_team_levels = []

        for participantFrameID, participantFrame in frame["participantFrames"].items():
            participant_id = participantFrame.get("participantId")
            level = participantFrame.get("level")
            # Xác định đội (100 = blue, 200 = red)
            if participant_id <= 5:  # Đội xanh
                blue_team_levels.append(level)
            else:  # Đội đỏ
                red_team_levels.append(level)

        # Lưu trữ cấp độ của các người chơi trong từng đường
        for lane, participants in lane_participants.items():
            key = f"level_difference_{lane}_at_{minute}"  # minute + 1 vì phút bắt đầu từ 0
            level_diff[key] = blue_team_levels[participants[0] - 1] - red_team_levels[participants[1] - 6]
    return level_diff


In [10]:
# Dict chênh lệch cấp độ mỗi đường, mỗi phút
level_difference_by_lane_df = calculate_level_difference_by_lane(data)
level_difference_by_lane_df

{'level_difference_TOP_at_0': 0,
 'level_difference_JUNGLE_at_0': 0,
 'level_difference_MID_at_0': 0,
 'level_difference_BOT_at_0': 0,
 'level_difference_SUPPORT_at_0': 0,
 'level_difference_TOP_at_1': 0,
 'level_difference_JUNGLE_at_1': 0,
 'level_difference_MID_at_1': 0,
 'level_difference_BOT_at_1': 0,
 'level_difference_SUPPORT_at_1': 0,
 'level_difference_TOP_at_2': 0,
 'level_difference_JUNGLE_at_2': -1,
 'level_difference_MID_at_2': 0,
 'level_difference_BOT_at_2': 0,
 'level_difference_SUPPORT_at_2': 0,
 'level_difference_TOP_at_3': 0,
 'level_difference_JUNGLE_at_3': -1,
 'level_difference_MID_at_3': 1,
 'level_difference_BOT_at_3': 0,
 'level_difference_SUPPORT_at_3': 1,
 'level_difference_TOP_at_4': 0,
 'level_difference_JUNGLE_at_4': -1,
 'level_difference_MID_at_4': 0,
 'level_difference_BOT_at_4': 0,
 'level_difference_SUPPORT_at_4': 2,
 'level_difference_TOP_at_5': 0,
 'level_difference_JUNGLE_at_5': 0,
 'level_difference_MID_at_5': 0,
 'level_difference_BOT_at_5': 0,
 '

### 4. Trạng thái quái khủng, mỗi đội, mỗi phút

In [ ]:
# Hàm tính chênh lệch quái khủng mỗi loại, mỗi phút giữa đội xanh và đỏ
def calculate_elite_monster_diff(data):
    initial_elite_monster_diff = {
        "HORDE": 0,
        "RIFTHERALD" : 0,
        "FIRE_DRAGON": 0,
        "WATER_DRAGON": 0,
        "EARTH_DRAGON": 0,
        "AIR_DRAGON": 0,
        "CHEMTECH_DRAGON": 0,
        "HEXTECH_DRAGON": 0,
        "ELDER_DRAGON": 0,
        "BARON_NASHOR" : 0
        
    }

    # Kết quả lưu trạng thái trụ theo từng phút
    elite_monster_status = {}

    # Lấy danh sách các frame (tương ứng phút)
    frames = data.get("info", {}).get("frames", [])

    # Duyệt qua từng frame (mỗi frame là 1 phút)
    for frame_index, frame in enumerate(frames):
        minute = frame_index

        for monster_type in initial_elite_monster_diff.keys():
            key = f"{monster_type}_diff_at_{minute}"
            elite_monster_status[key] = initial_elite_monster_diff[monster_type]

        # Lấy danh sách sự kiện trong frame
        events = frame.get("events", [])

        for event in events:
            # Kiểm tra nếu sự kiện là ăn quái
            if event.get("type") == "ELITE_MONSTER_KILL":
                team_id = event.get("killerTeamId")

                # Xác định đội (100 = blue, 200 = red)
                team = "blue" if team_id == 100 else "red"

                monster_type = event.get("monsterType", "")
                sub_monster_type = event.get("monsterSubType", "")

                if monster_type == "DRAGON":
                    if sub_monster_type:
                        key = f"{sub_monster_type}_diff_at_{minute}"
                        if team == "blue":
                            initial_elite_monster_diff[sub_monster_type] += 1
                        else:
                            initial_elite_monster_diff[sub_monster_type] -= 1
                        elite_monster_status[key] = initial_elite_monster_diff[sub_monster_type]
                else:
                    key = f"{monster_type}_diff_at_{minute}"
                    if team == "blue":
                        initial_elite_monster_diff[monster_type] += 1
                    else:
                        initial_elite_monster_diff[monster_type] -= 1
                    elite_monster_status[key] = initial_elite_monster_diff[monster_type]
    return elite_monster_status


In [16]:
# Dict chênh lệch quái khủng mỗi loại, mỗi phút giữa đội xanh và đỏ
elite_monster_diff = calculate_elite_monster_diff(data)
elite_monster_diff

{'HORDE_diff_at_0': 0,
 'RIFTHERALD_diff_at_0': 0,
 'FIRE_DRAGON_diff_at_0': 0,
 'WATER_DRAGON_diff_at_0': 0,
 'EARTH_DRAGON_diff_at_0': 0,
 'AIR_DRAGON_diff_at_0': 0,
 'CHEMTECH_DRAGON_diff_at_0': 0,
 'HEXTECH_DRAGON_diff_at_0': 0,
 'ELDER_DRAGON_diff_at_0': 0,
 'BARON_NASHOR_diff_at_0': 0,
 'HORDE_diff_at_1': 0,
 'RIFTHERALD_diff_at_1': 0,
 'FIRE_DRAGON_diff_at_1': 0,
 'WATER_DRAGON_diff_at_1': 0,
 'EARTH_DRAGON_diff_at_1': 0,
 'AIR_DRAGON_diff_at_1': 0,
 'CHEMTECH_DRAGON_diff_at_1': 0,
 'HEXTECH_DRAGON_diff_at_1': 0,
 'ELDER_DRAGON_diff_at_1': 0,
 'BARON_NASHOR_diff_at_1': 0,
 'HORDE_diff_at_2': 0,
 'RIFTHERALD_diff_at_2': 0,
 'FIRE_DRAGON_diff_at_2': 0,
 'WATER_DRAGON_diff_at_2': 0,
 'EARTH_DRAGON_diff_at_2': 0,
 'AIR_DRAGON_diff_at_2': 0,
 'CHEMTECH_DRAGON_diff_at_2': 0,
 'HEXTECH_DRAGON_diff_at_2': 0,
 'ELDER_DRAGON_diff_at_2': 0,
 'BARON_NASHOR_diff_at_2': 0,
 'HORDE_diff_at_3': 0,
 'RIFTHERALD_diff_at_3': 0,
 'FIRE_DRAGON_diff_at_3': 0,
 'WATER_DRAGON_diff_at_3': 0,
 'EARTH_DRA

### Làm sạch dữ liệu

In [14]:
def clean_df(df : pd.DataFrame):
    df = df.drop(columns=['metadata_dataVersion'] 
                + [col for col in df.columns if col.startswith('metadata_participants')]
                + [col for col in df.columns if col.startswith('info_participants')]
                )
    return df
